In [1]:
from collections import namedtuple
from map import Map

args = namedtuple('args', 'position distance type')

test_args = args(['London'], 3000, 'walk')
test_map = Map(test_args)

/home/quizznor/Desktop/machine-learning-from-scratch/travel_flood_map/.venv/lib/python3.12/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


In [ ]:
test_map.draw()

In [9]:
import networkx

times = list(networkx.single_source_dijkstra_path_length(test_map.graph, test_map.center_node, weight='travel_time').values())[-1]

In [10]:
print(times)

354.3
